# Importanto Bibliotecas & Definindo Constantes

In [ ]:
from typing import Dict, List
import os
import re
from collections import Counter

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

%matplotlib inline

In [ ]:
DATA_INPUT_FOLDER = 'data'
ARCHIVE_FOLDER = 'archive'
OUTPUT_FOLDER = 'output'

# Importando os Dados

Extraídos da base: https://www.kaggle.com/datasets/patkle/metacritic-scores-for-games-movies-tv-and-music

Os dados são referentes a notas de jogos, filmes, músicas e séries (de TV) avaliados por usuários e pelo próprio domínio Metacritic.

In [ ]:
# Um dicionário contendo os 4 DataFrames, separados 
df_dict : Dict[ str, pd.DataFrame ] = {}
for file_data in os.listdir(DATA_INPUT_FOLDER):
    df_dict[file_data[:-4]] = pd.read_csv(
        os.path.join( DATA_INPUT_FOLDER, file_data ),
        index_col=0,
        parse_dates= ['release_date'],
        dayfirst= True
    )

print(df_dict.keys())

# Analizando Inicialmente a Base exclusiva de Jogos

## Visão Geral

In [ ]:
games_df = df_dict['games']

# Observar informações gerais sobre os dados e tipos
display(games_df.info())
display(games_df.head())

O banco apresenta 20022 registros (linhas), com 7 propriedades (colunas):

- metascore: nota da plataforma
- plataforma
- data de lançamento
- número de ordenação (não está claro a que se refere)
- resumo: descrição breve do jogo
- título: nome do jogo
- nota dos usuários

Podemos verificar que 

- Há dados nulos (faltando) para o resumo (summary) de alguns registros
- Os dados de nota dos usuários (user_score) estão sendo tratados como texto (object) em vez de números (float)

## Entendendo o Signifado dos Dados

### sort_no

Analisar número de ordenação (sort_no), para tentar entendê-lo e descobrir se ele é útil

In [ ]:
# Conferir se é único
np.any(games_df['sort_no'].duplicated())

In [ ]:
# Conferir se tem alguma relação com outra coluna, principalmente de notas
games_df.sort_values('sort_no')

In [ ]:
# comparar se a ordenação pelo metascore (decrescente) e pelo sort_no (crescente) tem mesmas notas metascore
np.all(
    games_df.sort_values('metascore', ascending= False, ignore_index= True)['metascore'] == 
    games_df.sort_values('sort_no', ignore_index= True)['metascore']
)

Logo concluímos que sort_no é uma ordenação de ranking com base nos maiores metascore

## Levantando Hipóteses e Questionamentos

1. Qual o top 10 jogos mais bem avaliados pelo site? E pelos usuários?
2. Qual a plataforma que mais aparece entre os 100 melhores avaliados pelos usuários?
3. As notas dos jogos melhoraram a cada ano? E para cada plataforma ao longo dos anos?
4. Tem alguma epoca do ano que apresenta maior sucesso em relação a notas maiores?
5. Que palavras mais aparecem nos títulos dos 1000 melhores jogos? E dos 100 piores? E nos resumos?

## Limpando dados

### user_score

In [ ]:
# tentativa de converter o tipo dos dados de user_score, para ver os valores que falham
set_errors = set()
for score in games_df['user_score']:
    try:
        float(score)
    except Exception as e:
        set_errors.add(str(e))
print(set_errors)

In [ ]:
# Entendendo os registros com nota tbd (To Be Determined, traduzido como "a ser determinado")
display( games_df[ games_df['user_score'] == 'tbd' ].head() )
print(f"Número de registros com nota dos usuários pendente: { np.sum( games_df['user_score'] == 'tbd' ) }")
print(f"Porcentagem de registros com nota dos usuários pendente: { np.sum( games_df['user_score'] == 'tbd' ) / len( games_df ) :0.1%}")

Uma possível explicação para esse valor é de que esse número é uma média e ainda não tiveram avaliações de usuários o suficiente para computar uma média adequada.

Podemos adotar algumas abordagens:

- substituir os valores pela média geral
- separar em grupos de acordo com algum critério e substituir os valores pelas médias de cada grupo 
    - mesmo metascore
    - mesma plataforma
    - mesmo metascore e plataforma
- substituir os valores usando técnicas que mantenham a distribuição dos dados válidos
    - bfill (backward fill) : substituir pela proxima observacao
    - ffill (forward fill) : substituir pela observacao anterior
- excluir linhas

Como esses dados representam 7% dos valores, não desejo excluí-los, então tentarei uma média de acordo o agrupamento pela plataforma

In [ ]:
# para comparação
games_df['user_score_raw'] = games_df['user_score']

# Transformar a coluna em float, e os valores faltantes em NaN para poder computar a média
games_df['user_score'] = games_df['user_score'].replace('tbd', np.nan).astype(float)

# Criar uma cópia do dataframe, para alterar o user_score somente na cópia
games_df['user_score'] = games_df['user_score'].fillna(
    games_df.groupby('platform')['user_score'].transform('mean')
)

display( games_df[['user_score', 'user_score_raw']].head() )
display( games_df[games_df['user_score_raw'] == 'tbd'][['platform', 'user_score', 'user_score_raw']].head(10) )

games_df = games_df.drop(columns= 'user_score_raw')

In [ ]:
display(games_df.info())

### summary

In [ ]:
print('número de resumos faltantes:', games_df['summary'].isna().sum() )
display( games_df[games_df['summary'].isna()].head() )

Parece ser um caso de dados não coletados, pois a falta de um texto no csv gera um NaN

Podemos adotar algumas abordagens:

- procurar os dados na internet e preencher manualmente
- substituir por um valor padrão
- deixar como está
- excluir linhas

Como é uma descrição do jogo, não há necessidade em perder os outros dados por conta dele.

Substituirei os valores por uma string vazia, assim como está em games.csv

In [ ]:
games_df['summary'] = games_df['summary'].fillna('')
games_df['summary'].isna().sum()

## Funcoes auxiliares

Com objetivo de estudar palavras presentes na descrição e título, criarei uma função que realiza a extração das palavras

In [ ]:
def extract_words(texto : str) -> List[str]:
    return re.findall('\w+', texto)

extract_words('Injustice 2: Legendary Edition')

In [ ]:
def count_words(serie_texto : pd.Series) -> Dict[str, int]:
    '''
    It counts the number of text elements from the input series which contain a certain word
    @input:
        - serie_texto: a string-type pd.Series
    @output:
        A dict where each key is a word and the value its count as described
    '''
    word_counter = Counter()
    for texto in serie_texto:
        word_counter.update( Counter( set( extract_words(texto) ) ) )
    return word_counter

count_words(pd.Series(['Injustice 2: Legendary Edition', 'Injustice 2: Legendary Edition', 'Mini Metro']))

## EDA

### Análise Univariada

#### metascore